In [1]:
!nvidia-smi

Sat Mar  9 11:06:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers -U
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.5 MB/s eta 0:00:00


In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
# !pip install vllm

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
from tqdm import tqdm
import time
# from vllm import VLLM, SamplingParams

In [23]:
def load_json(path):
    with open(path, 'r', encoding="utf-8") as f: return json.load(f)

In [17]:
# model_name = "pphuc25/poem-vistral"
model_name = "Viet-Mistral/Vistral-7B-Chat"

system_prompt = "Bạn là một nhà thơ chuyên nghiệp, nhiệm vụ của bạn là chuyển bài văn này thành 1 bài thơ 5 chữ từ khoảng 1 đến 3 khổ"
# human = "Tấm nghe lời mẹ nên dẫn trâu đi thật xa. Ở nhà mẹ con Cám ra giếng gọi y như Tấm hay gọi mỗi ngày, cá nghe tiếng trồi lên miệng giếng, hai mẹ con Cám bắt bống đem làm thịt. Đến chiều chăn trâu về, Tấm đem cơm ra giếng kêu mà không thấy Bống lên, chỉ thấy nổi lên một cục máu đỏ. Thấy vậy Tấm ngồi khóc nức nở, Bụt hiện lên và hỏi:"

In [18]:
tokenizer1 = AutoTokenizer.from_pretrained(model_name)
model1 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16, # change to torch.float16 if you're using V100
    device_map="auto",
    use_cache=True,
)

tokenizer_config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [24]:
data_output = []
total_time = 0
for data in tqdm(load_json("/content/testset.json")):
    conversation = [{"role": "system", "content": system_prompt }]
    conversation.append({"role": "user", "content": data['text']})

    input_ids = tokenizer1.apply_chat_template(conversation, return_tensors="pt").to(model1.device)
    start_time = time.time()
    out_ids = model1.generate(
        input_ids=input_ids,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.95,
        top_k=20,
        temperature=0.1,
        repetition_penalty=1.05,
    )

    total_time += time.time() - start_time

    assistant = tokenizer1.batch_decode(out_ids[:, input_ids.size(1): ], skip_special_tokens=True)[0].strip()
    print("Assistant: ", assistant)
    output_dict = {"id": data["id"],
                   "text": data["text"],
                   "poem": assistant}
    data_output.append(output_dict)

print(total_time / 118)

with open('output.json', 'w') as f:
  json.dump(data_output, f)

  0%|          | 0/118 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 1/118 [00:05<11:28,  5.88s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Assistant:  Ngày xửa ngày xưa,
Một con gà mái với một cái mào đỏ.
Cô ấy sẽ soi gương,
Và hát một giai điệu vui vẻ,
"Cục ta cục tác! Mào tôi đã mọc!"

Mọi người đều ngưỡng mộ,
Cái mào đỏ tươi của cô ấy,
Họ nói, "Gà Mơ thật đáng yêu."
Với chiếc mũ đội đầu của cô ấy.

Vậy nên, mỗi ngày,
Cô ấy sẽ hát bài ca của mình,
"Cục ta cục tác! Mào tôi đã mọc!"
Và mọi người sẽ dừng lại để xem.


  2%|▏         | 2/118 [00:14<14:37,  7.56s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Assistant:  Bên hồ nước, một cây mơ nhỏ
Nghe tiếng khóc, nó dừng lại và nhìn quanh.
Một cây màu đỏ tía, lá thon dài
Đang khóc lóc, thật đáng thương biết bao!

Mơ thấy bạn buồn, nên đã hỏi thăm
"Chuyện gì xảy ra với anh vậy?"
Cây trả lời, "Các cây khác đều có hoa
Nhưng tôi thì không, tôi cảm thấy rất đau đớn."

Mơ nghĩ, "Tôi phải giúp đỡ người bạn này!"
Và nói, "Đừng lo, tôi sẽ tìm cách cho bạn."
Với sự giúp đỡ của gió, họ bắt đầu cuộc hành trình
Để tìm kiếm những bông hoa cho cây màu đỏ tía.

Họ đi qua đồng cỏ và rừng rậm
Tìm kiếm những bông hoa đẹp nhất, sáng nhất.
Cuối cùng, họ tìm thấy một bông hoa xinh đẹp
Có thể nở trên cây màu đỏ tía.

Mơ đưa


  3%|▎         | 3/118 [00:23<15:33,  8.12s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Assistant:  Trong rừng sâu, một ánh sáng lóe lên,
Lê Lợi kinh ngạc, trái tim anh ta đập mạnh.
Anh leo lên cao, tìm kiếm nguồn gốc của nó,
Và tìm thấy một chuôi gươm với một viên ngọc quý.

Trong tâm trí anh, một ký ức hiện ra,
Về một thanh gươm phát sáng ở nhà Thận.
Với sự tò mò, anh mang theo chuôi gươm,
Để xem liệu chúng có thể khớp với nhau hay không.

Một vài ngày sau, họ gặp nhau,
Lê Lợi chia sẻ câu chuyện của mình, và yêu cầu một lưỡi dao.
Lê Thận đưa cho anh ta lưỡi gươm cũ của mình,
Và khi đặt vào chuôi gươm, nó đã phù hợp hoàn hảo.

Ánh sáng rực rỡ, lưỡi dao sắc bén,
Một vũ khí để chiến đấu chống lại kẻ thù.


 99%|█████████▉| 117/118 [13:09<00:06,  6.24s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Assistant:  Trong thời kỳ khó khăn,
Lê Lợi đã đứng lên.
Với thanh gươm thiêng liêng,
Ông ấy đã thề sẽ dẫn dắt.

Nghĩa quân nổi dậy,
Trăm trận trăm thắng.
Quân Minh suy yếu,
Vào rừng họ không thể trốn thoát.

Kho lương thực đầy đủ,
Tinh thần chiến đấu tăng lên.
Họ đã chiến đấu dũng cảm,
Để bảo vệ đất đai của chúng ta.


100%|██████████| 118/118 [13:17<00:00,  6.76s/it]

Assistant:  Giặc tan, vua Lê lên ngôi,
Dân chúng vui mừng, đất trời yên bình.
Lê Lợi đã chiến thắng,
Trên ngai vàng, ông cai trị với sự khôn ngoan.

Một năm sau, một cảnh tượng kỳ lạ,
Rùa vàng nổi lên, tìm kiếm thanh kiếm.
"Thưa bệ hạ," nó nói, "Tôi nhớ,
Long Quân đã cho anh mượn thanh kiếm thần thánh.

Nhà vua ngạc nhiên, nhưng không hề sợ hãi,
Ông ta rút thanh kiếm ra khỏi vỏ.
Con rùa vươn cổ, nắm chặt lưỡi kiếm,
Và biến mất vào sâu thẳm của hồ.

Vua Lê nhìn theo, đầy kinh ngạc,
Khi con rùa mang thanh kiếm trở lại.
Từ đó, đất nước được an toàn,
Dưới sự cai trị công bằng của vua Lê.
6.754121933953237


In [ ]:
# template = [
#     {
#         "role": "system",
#         "content": """Bạn là một nhà kể chuyện phiếm, nhiệm vụ của bạn là hãy kể 1 câu chuyện đơn giản và ngắn gọn từ một bài thơ, câu chuyện nên là 1 bài liền mạch, thực tế"""
#     }
# ]

# llm = LLM(model=args.model)
# sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=1024)
# tokenizer = AutoTokenizer.from_pretrained(args.model, device_map="auto")

# def apply_template(data):
#     poem = data["poem"]
#     data["prompts_vllm"] = tokenizer.apply_chat_template(template + [{"role": "user", "content": poem}], tokenize=False)
#     return data

# prompts = data.map(apply_template, num_proc=args.num_proc)


# count_file = 1
# for idx in tqdm(range(0, len(prompts), args.batch_size)):
#     final_data = []
#     if idx + args.batch_size >= len(prompts):
#         end = len(prompts)
#     else:
#         end = idx+args.batch_size
#     outputs = llm.generate(prompts[idx:end]["prompts_vllm"], sampling_params)

#     for output, prompt_origin in zip(outputs, prompts[idx:end]["prompts_vllm"]):
#         final_data.append({"text": output.outputs[0].text.strip(), "prompt": prompt_origin})


In [ ]:
# load_json("/content/testset.json")

In [ ]:
# output_dict = []
# for i,j in zip(data_output, load_json("/content/testset.json", encoding='utf-8')):
#     output = {}
#     output['id'] = j['id']
#     output['text'] = j["id"]
#     output['poem'] = i
#     output_dict.append(output)
# # Save to json file
# with open('output.json', 'w') as f:
#     json.dump(output_dict, f)

In [14]:
# Gradio
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 11.2 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl

In [16]:
import gradio as gr
import os

def summarize(x):
    system_prompt = "Bạn là một nhà thơ chuyên nghiệp, nhiệm vụ của bạn là chuyển bài văn này thành 1 bài thơ 5 chữ từ khoảng 1 đến 3 khổ"
    conversation = [{"role": "system", "content": system_prompt }]
    conversation.append({"role": "user", "content": x})

    input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(model.device)
    start_time = time.time()
    out_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.95,
        top_k=20,
        temperature=0.1,
        repetition_penalty=1.05,
    )

    assistant = tokenizer.batch_decode(out_ids[:, input_ids.size(1): ], skip_special_tokens=True)[0].strip()
    return assistant

gr.close_all()
demo = gr.Interface(fn=summarize,
                    inputs=[gr.Textbox(label="Text to generate poem", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=5)],
                    title="Poem generation with VistralPoem",
                    description="Summarize any text with poem",
                    examples=["Tấm nghe lời mẹ nên dẫn trâu đi thật xa. Ở nhà mẹ con Cám ra giếng gọi y như Tấm hay gọi mỗi ngày, cá nghe tiếng trồi lên miệng giếng, hai mẹ con Cám bắt bống đem làm thịt. Đến chiều chăn trâu về, Tấm đem cơm ra giếng kêu mà không thấy Bống lên, chỉ thấy nổi lên một cục máu đỏ. Thấy vậy Tấm ngồi khóc nức nở, Bụt hiện lên và hỏi:",
                              "Trong suốt chiều 9.3, Công ty Bảo Tín Minh Châu liên tục tăng giá vàng nhẫn với mỗi lần từ 40.000 - 60.000 đồng mỗi lượng, giá mua vào của công ty này lên 69,73 triệu đồng, bán ra 71,03 triệu đồng. Chỉ chưa đầy 2 ngày qua, Công ty Bảo Tín Minh Châu đã tăng giá vàng nhẫn 2,15 triệu đồng mỗi lượng và trở thành đơn vị có mức giá vàng nhẫn cao nhất trên thị trường hiện nay. Kế đến là Tập đoàn Doji có giá bán vàng nhẫn cao thứ 2, ở mức 70,9 triệu đồng/lượng, chiều mua vào ở mức 69,6 triệu đồng/lượng. Tập đoàn Doji liên tục tăng giá vàng nhẫn với mỗi bước nhảy lên từ 100.000 - 200.000 đồng mỗi lượng, nâng tổng mức tăng lên 1,3 triệu đồng/lương so với mở cửa đầu ngày. Trong khi đó, Công ty vàng bạc đá quý Sài Gòn - SJC tăng giá vàng nhẫn gàn 1,5 triệu đồng/lượng trong 2 ngày qua, giá mua vào lên 68,35 triệu đồng, bán ra 69,7 triệu đồng…"]
                   )
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://432ec0ebbf6e4b8b17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
